In [1]:
b

NameError: name 'b' is not defined

- Random
- Bandit Algorithms
    * contextual
    * optimistic
    * UCB
    * Gradient
- Epsilon Greedy
- Policy Iteration/Dynamic Programming (Bellman Equations)
    * policy iteration
    * value iteration

- Monte Carlo
    * exploring starts
    * first visit mc 
    * importance sampling
* 
- Temporal Dynamics (TD0)
- Sarsa
- Expected Sarsa


In [260]:
import numpy as np
import random

In [285]:
char_lookup = {-1:'X', 1:'Y', 0:' '}
def rotate_array(a): return np.flip(a.T,1)
def flip_horizontal(a): return np.flip(a,0)
def flip_vertical(a): return np.flip(a,1)

class State():
    #__slots__=
    def __init__(self, data, win): #update_function):
        #self.update_function=update_function
        self.data, self.win=data,win
        self._actions = []
        self._action_probabilities = []
        self._action_rewards = []
        
        self._successor_states = []
        
    def copy_data(self): return self.data.copy()    
    def get_move(self):  return self._chose_action()
    def is_win(self): return self.win
    def __repr__(self):  return '\n'.join([' | '.join([char_lookup.get(i,'U') for i in row]) for row in self.data])

    


class Board():
    def __init__(self, size=3):
        self.size = size
        self.state = State(np.zeros((size,size)), False)
        self.all_states = {}
        self._initialize_states(self.state, 1)
    def __repr__(self): return self.state.__repr__()
    def state_to_string(self, data): return ''.join([''.join([char_lookup.get(i,'U') for i in row]) for row in data]) 
    def find_state(self, data):
        """Return the relevant state or None if it is not found"""
        #check rotations
        for _ in range(4):
            state_string = self.state_to_string(data)
            if state_string in self.all_states: return self.all_states[state_string]
            data = rotate_array(data)
        #check flip and rotations
        data = flip_horizontal(data)
        for _ in range(4):
            state_string = self.state_to_string(data)
            if state_string in self.all_states: return self.all_states[state_string]
            data = rotate_array(data)
        #return None if not found    
        return None
    def _initialize_states(self, starter_state, player):
        if starter_state.win: return
        
        for coordinates in self.get_moves(starter_state):
            new_state = starter_state.copy_data()
            new_state[coordinates] = player
            if self.find_state(new_state) is None:
                state_string = self.state_to_string(new_state)
                win = self._check_win(new_state)
                new_state = State(new_state, win)
                self.all_states[state_string]=new_state
                self._initialize_states(new_state, player*-1)  
        return

    def get_moves(self, state=None): 
        if state is None: state = self.state
        return list(zip(*np.where(state.copy_data()==0)))
    def _check_win(self, data):
        #check rows and columns
        data = np.array(data)
        for axis in [0,1]:
            if np.any(np.abs(np.sum(data, axis=axis))==self.size): return True
        #check diagonal (kept seperate for clarity)
        for axis in [0,1]:    
            if np.abs(data.diagonal(axis).sum())==self.size: return True
        #otherwise return false
        return False
    
    def is_win_state(self): return self.state.win
    def move(self, player, coordinates):
        data = self.state.copy_data()
        data[coordinates] = player
        new_state = self.find_state(data)
        assert new_state is not None, f"Invalid state loaded {data}"
        self.state = new_state
        return True
            
        
        
        
class Game():
    def __init__(self, player1, player2, board = None):
        self.board= Board() if board is None else board
        self.players = [(1,player1), (-1,player2)]
        
    def play(self, first_player=0):
        self.turn = 0
        for player_key, player in self.players: player.board = self.board
        while not self.is_win_state() and self.turn<self.board.size**2:
            self.turn +=1
            #Change Player
            player_key, player = self.players[(self.turn+first_player)%2]

            #Figure out the move
            coordinates = player.move()
            
            #Make the move
            self.board.move(player_key, coordinates)
            #print(self.state)
            
            #After move processing
            player.after_move()
            
        player.after_game()
    def __getattr__(self, k): return getattr(self.board, k)

    
        

class Player():
    def __init__(self, **kwargs): pass
    def move(self): raise NotImplementedError
    def after_move(self): return False
    def after_game(self): return False
    def __getattr__(self, k): return getattr(self.board, k)
    


class RandomMover(Player):
    def move(self): 
        all_moves = self.board.get_moves()
        #print(all_moves, type(all_moves))
        return random.choice(all_moves)
        
       


In [286]:
g = Game(RandomMover(), RandomMover())
g.play()
g.state

AssertionError: Invalid state loaded [[ 0.  0. -1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]

In [239]:
b=Board()

In [240]:
b.get_moves()

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

NameError: name 'turn' is not defined

In [224]:
g.state

  |   |  
  |   |  
  |   |  

In [171]:
for state in b.all_states.values():
    if state.win: print(state, '\n')

Y | X | Y
X | Y | X
Y | X | Y 

Y | X | Y
X | Y | Y
X | X | Y 

Y | X | Y
X | Y |  
X |   | Y 

Y | X | Y
X | Y |  
  | X | Y 

Y | X | Y
X | X | Y
Y | X |   

Y | X | Y
X | X | Y
X | Y | Y 

Y | X | Y
X | X | Y
  |   | Y 

Y | X | Y
X |   | Y
X |   | Y 

Y | X | Y
X |   | Y
  | X | Y 

Y | X | Y
X | X | X
Y |   | Y 

Y | X | Y
Y | X | Y
X | X |   

Y | X | Y
Y | X |  
X | X | Y 

Y | X | Y
Y | X |  
  | X |   

Y | X | Y
Y | X |  
Y |   | X 

Y | X | Y
  | X |  
Y | X |   

Y | X | Y
Y | Y |  
X | X | X 

Y | X | Y
Y |   | Y
X | X | X 

Y | X | Y
  | Y |  
X | X | Y 

Y | X | Y
  |   | Y
X | X | Y 

Y | X | X
Y | Y | X
Y |   |   

Y | X | X
Y | Y | X
X | Y | Y 

Y | X | X
Y | Y | X
  | Y | X 

Y | X | X
Y | Y | X
  |   | Y 

Y | X | X
Y | Y | Y
X |   |   

Y | X | X
Y | Y | Y
X | Y | X 

Y | X | X
Y | Y |  
X |   | Y 

Y | X | X
Y | Y | Y
  | X |   

Y | X | X
Y | Y |  
Y | X |   

Y | X | X
Y | Y |  
  | X | Y 

Y | X | X
Y | Y | Y
  |   | X 

Y | X | X
Y | Y |  
Y |   | X 

Y | X | 

In [172]:
for key, state in b.all_states.items(): 
    print(key)

Y        
YX       
YXY      
YXYX     
YXYXY    
YXYXYX   
YXYXYXY  
YXYXYXYX 
YXYXYXYXY
YXYXYXY X
YXYXYXYYX
YXYXYX Y 
YXYXYXXY 
YXYXY X  
YXYXYYX  
YXYXYYXX 
YXYXYYXXY
YXYXYYX X
YXYXYYXYX
YXYXY XY 
YXYXY XYX
YXYXY X Y
YXYXY  X 
YXYXYY X 
YXYXYY XX
YXYXY  XY
YXYXY   X
YXYXYY  X
YXYXY Y X
YXYXY  YX
YXYX Y   
YXYXXY   
YXYXXYY  
YXYXXYYX 
YXYXXYY X
YXYXXYYYX
YXYXXY Y 
YXYXXYXY 
YXYXXYXYY
YXYXXY YX
YXYXXY  Y
YXYX YX  
YXYX YXY 
YXYX YXYX
YXYX YX Y
YXYX Y X 
YXYX YYX 
YXYX YYXX
YXYX Y XY
YXYX Y  X
YXYX YY X
YXYX Y YX
YXYX  Y  
YXYXX Y  
YXYXX Y Y
YXYXXXY Y
YXYX XY  
YXYX XY Y
YXYX XYXY
YXYX  Y X
YXYX   Y 
YXYXX  Y 
YXYXX  YY
YXYXX XYY
YXYX X Y 
YXYX  XY 
YXYX  XYY
YXYX   YX
YXYX    Y
YXYXX   Y
YXYX  X Y
YXYX   XY
YXY X    
YXYYX    
YXYYX X  
YXYYXYX  
YXYYXYXX 
YXYYXYX X
YXYYXYXYX
YXYYX XY 
YXYYX XYX
YXYYX X Y
YXYYX XXY
YXYYX  X 
YXYYX   X
YXYYX Y X
YXYYX  YX
YXY X Y  
YXY X YX 
YXY X Y X
YXY X YYX
YXY X  Y 
YXY X XY 
YXY   X  
YXYY  X  
YXYY  XX 
YXYYY XX 
YXYYY XXX
YXYY YXX 
YXYY YXXX


In [80]:
b.check_win(np.eye(3))

True

In [106]:
arr1 = np.eye(3)

In [107]:
arr2=arr1.copy()

In [108]:
arr2[1,2]=5

In [109]:
arr1, arr2

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]), array([[1., 0., 0.],
        [0., 1., 5.],
        [0., 0., 1.]]))

In [49]:
b.find_state(no.zeros())

In [50]:
b.all_states

{}

In [45]:
State.state_to_string(arr)

' YUUUUUUU'

In [17]:
new_arr = arr
for _ in range(4):
    print(new_arr, '\n')
    new_arr=rotate_array(new_arr)

new_arr = flip_horizontal(arr)
for _ in range(4):
    print(new_arr, '\n')
    new_arr=rotate_array(new_arr)

[[0 1 2]
 [3 4 5]
 [6 7 8]] 

[[6 3 0]
 [7 4 1]
 [8 5 2]] 

[[8 7 6]
 [5 4 3]
 [2 1 0]] 

[[2 5 8]
 [1 4 7]
 [0 3 6]] 

[[6 7 8]
 [3 4 5]
 [0 1 2]] 

[[0 3 6]
 [1 4 7]
 [2 5 8]] 

[[2 1 0]
 [5 4 3]
 [8 7 6]] 

[[8 5 2]
 [7 4 1]
 [6 3 0]] 

[[2 1 0]
 [5 4 3]
 [8 7 6]] 

[[8 5 2]
 [7 4 1]
 [6 3 0]] 

[[6 7 8]
 [3 4 5]
 [0 1 2]] 

[[0 3 6]
 [1 4 7]
 [2 5 8]] 



In [34]:
char_lookup = {-1:'X', 1:'O', 0:' '}
chars = '\n'.join([' | '.join([char_lookup.get(i,'U') for i in row]) for row in arr])
print(chars)

  | O | U
U | U | U
U | U | U


In [6]:
arr = np.array(list(range(9))).reshape(3,3)

In [4]:
char_lookup.get()

NameError: name 'arr' is not defined

In [57]:
np.any(np.abs(np.sum(arr, axis=1))==3)

True

In [15]:
flip_horizontal(arr)

array([[6, 7, 8],
       [3, 4, 5],
       [0, 1, 2]])

In [90]:
list(zip(*np.where(np.zeros((3,3))==0)))

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [62]:
arr

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])